In [2]:
import re
import pandas as pd
import numpy as np
# from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [327]:
# 17개 지역
regions = {'11': '서울', '12': '부산', '14': '인천', '16': '대전', '15': '광주',
'13': '대구', '17': '울산', '27': '세종', '18': '경기', '19': '강원특별자치도',
'20': '충북', '21': '충남', '22': '전북특별자치도', '23': '전남', '24': '경북',
'25': '경남', '26': '제주'}


# 수도권 : 서울, 인천, 세종, 경기
# 수도권 외 광역시: 부산, 대구, 광주, 대전, 울산
# 그 외 지방: 강원, 충북, 충남, 전북, 전남, 경북, 경남, 제주

features = [
    "region",
    "cpname",
    "title",
    "career",
    "education",
    "jobtype",
    "cptype",
    "sales",
    "employees",
    "aversalary",
    "capital",
    "pros",
]
df = pd.DataFrame(columns=features)

for region in regions.values():
    df_subset = pd.read_csv(f'sample data/raw_data_{region}.csv', index_col=0)
    df = pd.concat([df, df_subset])

df = df.reset_index(drop=True)

df.to_csv("raw_data.csv", encoding="utf-8-sig")

In [328]:
df = pd.read_csv('raw_data.csv', encoding='utf-8-sig', index_col=0)

In [329]:
df.drop('cpname', axis=1, inplace=True)
df.drop('title', axis=1, inplace=True)
df.drop('career', axis=1, inplace=True)
df.head()

,region,education,jobtype,cptype,sales,employees,aversalary,capital,pros
0,서울,대졸이상,정규직,중견기업,"1조 8,090억 8,204만원 (2023)","2,036 명 재직중","9,516만원\n자세히보기","778억 1,959만원",안정적인 장수 회사-성장판 열린 회사-구직자 인기기업-복지의 끝판왕-매출 1조 클럽...
1,서울,학력무관,인턴,대기업,"6조 885억 8,200만원 (2023)","1,657 명 재직중","5,083만원\n자세히보기","6,486억 5,400만원",안정적인 장수 회사-성장판 열린 회사-복지의 끝판왕-매출 1조 클럽-아침을 왜 굶어...
2,서울,학력무관,인턴,중기업,"113조 7,600억 976만원 (2022)","15,761 명 재직중","6,176만원\n자세히보기",-,성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-매출 1조 클럽-HOT한 인...
3,서울,학력무관,정규직,중견기업,"25조 4,700억원 (2022)","18,000 명 재직중",-,-,안정적인 장수 회사-구직자 인기기업-채용에 적극적인 회사-HOT한 인기기업
4,서울,학력무관,정규직외,중견기업,"6,569억 5,188만원 (2022)","2,610 명 재직중","6,244만원\n자세히보기","2조 3,577억 6,610만원",성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-HOT한 인기기업-집 걱정 ...


In [330]:
print(df['education'].value_counts())
print(df['jobtype'].value_counts())
print(df['cptype'].value_counts())

education
학력무관     1779
고졸이상      398
대졸이상      354
초대졸이상     291
고졸         56
대졸         52
초대졸        44
석사이상       25
박사이상       14
석사          1
박사          1
Name: count, dtype: int64
jobtype
정규직         1455
계약직          462
정규직외         451
위촉직          158
위촉직외         134
인턴            88
계약직외          86
알바            66
프리랜서          55
교육생(연수생)      50
파견직            5
알바외            2
인턴외            1
병역특례           1
대체인력           1
Name: count, dtype: int64
cptype
중소기업    1444
중견기업     675
대기업      673
소기업      152
중기업       69
무입력        2
Name: count, dtype: int64


In [331]:
# le = LabelEncoder()
# le.fit_transform()

df['education'] = df['education'].replace({
    '학력무관': 0,
    '고졸': 1,
    '고졸이상': 1,
    '초대졸' : 2,
    '초대졸이상': 2,
    '대졸': 3,
    '대졸이상': 3,
    '석사' : 4,
    '석사이상' : 4,
    '박사' : 5,
    '박사이상' : 5
},)

def encode_cptype(x : str):
    if x == '대기업':
        return 2
    elif x == '중견기업':
        return 1
    else:
        return 0
    

df['jobtype'] = df['jobtype'].map(lambda x : 1 if x == '정규직' else 0)

df['cptype'] = df['cptype'].map(encode_cptype)

C:\Users\user\AppData\Local\Temp\ipykernel_8104\1285458529.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['education'] = df['education'].replace({


In [332]:
print(df['education'].value_counts())
print(df['jobtype'].value_counts())
print(df['cptype'].value_counts())
df.head()

education
0    1779
1     454
3     406
2     335
4      26
5      15
Name: count, dtype: int64
jobtype
0    1560
1    1455
Name: count, dtype: int64
cptype
0    1667
1     675
2     673
Name: count, dtype: int64


,region,education,jobtype,cptype,sales,employees,aversalary,capital,pros
0,서울,3,1,1,"1조 8,090억 8,204만원 (2023)","2,036 명 재직중","9,516만원\n자세히보기","778억 1,959만원",안정적인 장수 회사-성장판 열린 회사-구직자 인기기업-복지의 끝판왕-매출 1조 클럽...
1,서울,0,0,2,"6조 885억 8,200만원 (2023)","1,657 명 재직중","5,083만원\n자세히보기","6,486억 5,400만원",안정적인 장수 회사-성장판 열린 회사-복지의 끝판왕-매출 1조 클럽-아침을 왜 굶어...
2,서울,0,0,0,"113조 7,600억 976만원 (2022)","15,761 명 재직중","6,176만원\n자세히보기",-,성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-매출 1조 클럽-HOT한 인...
3,서울,0,1,1,"25조 4,700억원 (2022)","18,000 명 재직중",-,-,안정적인 장수 회사-구직자 인기기업-채용에 적극적인 회사-HOT한 인기기업
4,서울,0,0,1,"6,569억 5,188만원 (2022)","2,610 명 재직중","6,244만원\n자세히보기","2조 3,577억 6,610만원",성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-HOT한 인기기업-집 걱정 ...


In [333]:
# 띄어쓰기, 콤마(',') 제거
for col in ['sales', 'aversalary', 'capital', 'employees']:
    df[col] = df[col].map(lambda x : x[:x.rfind('(')] if x.rfind('(') != -1 else x)
    df[col] = df[col].str.replace(',', '')
    df[col] = df[col].str.replace(' ', '')
    

# 빈 값이라면 '-' 으로 대체
df['sales'] = df['sales'].apply(lambda x : x if x != '' else '-')
df['aversalary'] = df['aversalary'].str.replace('\n자세히보기', '')
df['employees'] = df['employees'].map(lambda x : x[:x.rfind('명')])

In [334]:
df.head()

,region,education,jobtype,cptype,sales,employees,aversalary,capital,pros
0,서울,3,1,1,1조8090억8204만원,2036,9516만원,778억1959만원,안정적인 장수 회사-성장판 열린 회사-구직자 인기기업-복지의 끝판왕-매출 1조 클럽...
1,서울,0,0,2,6조885억8200만원,1657,5083만원,6486억5400만원,안정적인 장수 회사-성장판 열린 회사-복지의 끝판왕-매출 1조 클럽-아침을 왜 굶어...
2,서울,0,0,0,113조7600억976만원,15761,6176만원,-,성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-매출 1조 클럽-HOT한 인...
3,서울,0,1,1,25조4700억원,18000,-,-,안정적인 장수 회사-구직자 인기기업-채용에 적극적인 회사-HOT한 인기기업
4,서울,0,0,1,6569억5188만원,2610,6244만원,2조3577억6610만원,성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-HOT한 인기기업-집 걱정 ...


In [335]:
print(df['region'].value_counts())

region
서울         180
대구         180
경북         180
충북         180
강원특별자치도    180
경기         180
대전         179
전북특별자치도    179
인천         179
부산         177
충남         177
경남         177
제주         177
울산         176
세종         176
전남         174
광주         164
Name: count, dtype: int64


In [336]:
# 수도권 : 서울, 인천, 세종, 경기
# 수도권 외 광역시: 부산, 대구, 광주, 대전, 울산
# 그 외 지방: 강원, 충북, 충남, 전북, 전남, 경북, 경남, 제주

def encode_region(x : str):
    if x == '서울' or x =='인천' or x =='세종' or x =='경기':
        return 2
    elif x == '부산' or x == '대구'or x == '광주'or x =='대전' or x =='울산':
        return 1
    else:
        return 0
    

df['region'] = df['region'].map(encode_region)

In [337]:
print(df['region'].value_counts())

region
0    1424
1     876
2     715
Name: count, dtype: int64


In [338]:
tags_encoded = pd.read_csv('tags_encoded.csv', encoding='utf-8-sig', index_col=0)
# tags_encoded.head()
tags_score = dict(zip(tags_encoded['tags'], tags_encoded['score']))
tags_score

{'안정적인 장수 회사': 5,
 '성장판 열린 회사': 3,
 '구직자 인기기업': 5,
 '채용에 적극적인 회사': 3,
 '복지의 끝판왕': 1,
 '매출 1조 클럽': 5,
 '매출 상위 10%': 5,
 'HOT한 인기기업': 5,
 '아침을 왜 굶어요?': 1,
 '평균연봉 6천이상': 3,
 '신입연봉이 4천이상': 3,
 '스타트업 초봉 3,500만원↗': 3,
 '지방에 위치한 대기업': 5,
 '자기계발비 지원 기업': 1,
 '보너스로 행복해요♥': 1,
 '내 출퇴근은 내 맘대로': 1,
 '아낌없이 주는 커피': 1,
 '아이와 함께 출근해요': 1,
 '잘나가는 의료·바이오기업': 5,
 '주 4일 근무': 3,
 '집 걱정 안 하기': 3,
 '거침없이 성장하는 회사': 3,
 '매출 1000대 기업': 5,
 '인크루트 인증 우수기업': 3,
 '대기업 in 서울': 5,
 '공기업 in 서울': 5,
 '장기근속수당제공 기업': 3,
 '육아휴직 제공 기업': 1,
 'KOSPI 상장': 5,
 'KOSDAQ 상장': 5,
 '빵빵한 구내식당': 1,
 '휴식까지 챙겨줘요!': 1,
 'KONEX 상장': 3}

In [339]:
def encodeTagsByScore(tags: str):
    if isinstance(tags, float): return tags
    total_score = 0
    for tag in tags.split('-'):
        if tag !=  '':
            total_score += tags_score[tag]
    return total_score

df['pros_encoded'] = df['pros'].apply(encodeTagsByScore)
df['pros_encoded']

0       50.0
1       31.0
2       29.0
3       18.0
4       32.0
        ... 
3010     NaN
3011     NaN
3012     NaN
3013     1.0
3014     3.0
Name: pros_encoded, Length: 3015, dtype: float64

In [340]:
nullable_features = ['sales',	'employees',	'aversalary',	'capital']

# '-' 값들 모두 np.nan 로 변경
for nullable_feature in nullable_features:
    # 값이 음수 (sales : - 15억 3,272만원 (2023) ) 인 경우도 있으므로 replace 함수가 아닌 map 함수 사용
    df[nullable_feature] = df[nullable_feature].map(lambda x : np.nan if x == '-' else x)

# np.nan 값 0 으로 변경
df['pros_encoded'] = df['pros_encoded'].replace(np.nan, 0)

# np.nan 값을 제외하고 float 로 형변환. (np.nan 는 float)
# 5월 4일 기준으로 수정(line 242)했지만, 수정 이전에는 crawling.py 파일에서 employees 의 결측값을 empty string 으로 저장
# 이미 크롤링한 raw_data.csv 파일 기준으로는 바로 아래 코드가 필요
df['employees'] = df['employees'].map(lambda x: float(x) if x != '' else np.nan)
# 수정된 버전
# df['employees'] = df['employees'].map(lambda x: float(x))

print(df['sales'].isna().sum())
print(df['employees'].isna().sum())
print(df['aversalary'].isna().sum())
print(df['capital'].isna().sum())
print(df['pros_encoded'].isna().sum())

print(df.dtypes)
print(type(df['sales'][0]))
print(type(df['aversalary'][0]))
print(type(df['capital'][0]))

df.head()

762
533
866
712
0
region            int64
education         int64
jobtype           int64
cptype            int64
sales            object
employees       float64
aversalary       object
capital          object
pros             object
pros_encoded    float64
dtype: object
<class 'str'>
<class 'str'>
<class 'str'>


,region,education,jobtype,cptype,sales,employees,aversalary,capital,pros,pros_encoded
0,2,3,1,1,1조8090억8204만원,2036.0,9516만원,778억1959만원,안정적인 장수 회사-성장판 열린 회사-구직자 인기기업-복지의 끝판왕-매출 1조 클럽...,50.0
1,2,0,0,2,6조885억8200만원,1657.0,5083만원,6486억5400만원,안정적인 장수 회사-성장판 열린 회사-복지의 끝판왕-매출 1조 클럽-아침을 왜 굶어...,31.0
2,2,0,0,0,113조7600억976만원,15761.0,6176만원,NaN,성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-매출 1조 클럽-HOT한 인...,29.0
3,2,0,1,1,25조4700억원,18000.0,NaN,NaN,안정적인 장수 회사-구직자 인기기업-채용에 적극적인 회사-HOT한 인기기업,18.0
4,2,0,0,1,6569억5188만원,2610.0,6244만원,2조3577억6610만원,성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-HOT한 인기기업-집 걱정 ...,32.0


In [341]:
def convertFromStr(input):
    # 단위 : 백만원
    # 띄어쓰기가 없다고 전제
    # input = input.replace(' ', '')
    # input : str or np.nan
    # type(np.nan) == float
    if type(input) != str: return input
    p = re.compile('(\d{1,4}조)?(\d{1,4}억)?(\d{1,4}만)?원')
    match = p.search(input)
    if (match is None) : 
        print(input) # '-'
        return input
        
    value = 0
    for group in match.groups():
        if group is None: # 마지막 group 은 None
            continue
        elif group[-1] == '조':
            value += int(group[:-1]) * 1000000
        elif group[-1] == '억':
            value += int(group[:-1]) * 100
        elif group[-1] == '만':
            value += int(group[:-1]) // 100
        else: 
            continue
    return value

for col in ['sales', 'aversalary', 'capital', 'capital']:
    # 단위 : 백만원
    df[col] = df[col].map(convertFromStr)

In [347]:
df.drop('pros', axis=1, inplace=True)
print(df.dtypes)
df.head()

region            int64
education         int64
jobtype           int64
cptype            int64
sales           float64
employees       float64
aversalary      float64
capital         float64
pros_encoded    float64
dtype: object


,region,education,jobtype,cptype,sales,employees,aversalary,capital,pros_encoded
0,2,3,1,1,1809082.0,2036.0,95.0,77819.0,50.0
1,2,0,0,2,6088582.0,1657.0,50.0,648654.0,31.0
2,2,0,0,0,113760009.0,15761.0,61.0,NaN,29.0
3,2,0,1,1,25470000.0,18000.0,NaN,NaN,18.0
4,2,0,0,1,656951.0,2610.0,62.0,2357766.0,32.0


In [348]:
df.to_csv('data_beforeScaling.csv', encoding='utf-8-sig')